In [1]:
from models import *
from utils import *
import torch
from torch import nn

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


### Step 1: Obtain the student and teacher models which allow feature map extraction

In [2]:
VGG16_teacher = vgg16_bn_f(False,100)
VGG16_teacher.load_state_dict(torch.load('teacher_model_vgg16_bn.pth'))

C:\Users\PCF\AppData\Local\Temp\ipykernel_26796\1514609959.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  VGG16_teacher.load_state_dict(torch.load('teacher_model_vgg16_

<All keys matched successfully>

In [3]:
VGG11_student = vgg11_bn_f(False,100)

Distill using CRD

In [4]:
crd_net = create_crd_model(VGG16_teacher, VGG11_student)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
crd_net.student = crd_net.student.to(device)
crd_net.teacher = crd_net.teacher.to(device)
crd_net.crd_block = crd_net.crd_block.to(device)
EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(get_learnable_parameters(crd_net), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_crd(crd_net,train_loader_idx,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(crd_net.student,test_loader,nn.CrossEntropyLoss(),device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))



Epoch: 1 Train Loss: 12.94 Train Accuracy: 0.03
Epoch: 2 Train Loss: 12.84 Train Accuracy: 0.04
Epoch: 3 Train Loss: 12.77 Train Accuracy: 0.05
Epoch: 4 Train Loss: 12.70 Train Accuracy: 0.07
Epoch: 5 Train Loss: 12.62 Train Accuracy: 0.08
Epoch: 6 Train Loss: 12.56 Train Accuracy: 0.10
Epoch: 7 Train Loss: 12.50 Train Accuracy: 0.11
Epoch: 8 Train Loss: 12.44 Train Accuracy: 0.13
Epoch: 9 Train Loss: 12.38 Train Accuracy: 0.15
Epoch: 10 Train Loss: 12.33 Train Accuracy: 0.17
Epoch: 11 Train Loss: 12.28 Train Accuracy: 0.19
Epoch: 12 Train Loss: 12.24 Train Accuracy: 0.21
Epoch: 13 Train Loss: 12.19 Train Accuracy: 0.23
Epoch: 14 Train Loss: 12.15 Train Accuracy: 0.25
Epoch: 15 Train Loss: 12.11 Train Accuracy: 0.27
Epoch: 16 Train Loss: 12.07 Train Accuracy: 0.28
Epoch: 17 Train Loss: 12.04 Train Accuracy: 0.30
Epoch: 18 Train Loss: 12.01 Train Accuracy: 0.32
Epoch: 19 Train Loss: 11.97 Train Accuracy: 0.33
Epoch: 20 Train Loss: 11.94 Train Accuracy: 0.35
Epoch: 21 Train Loss: 11.91 T

In [5]:
eval_f(crd_net.student,test_loader,nn.CrossEntropyLoss(),device)

(2.279502033428022, array(0.4204, dtype=float32))

In [9]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.47193998
Final Test Accuracy: 0.4188
Final Train Loss: 11.715296087972343
Final Test Loss: 2.2781109870619076


In [7]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_crd_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_crd_ts.csv")

In [10]:
torch.save(crd_net.student.state_dict(),"vgg11_cifar100_crd_student.pth")
torch.save(crd_net.crd_block.state_dict(),"vgg11_cifar100_crd_embedder.pth")